In [16]:
#def my_likelihood():
#    return


In [17]:
##Priors Info

# k1 = uniform(1.7*10**-9,1.06*10**-9)##...?
# GA1 = uniform(1.4,3.6)
# GA2 = uniform(0,8)
# GA3 = uniform(0.5,7.5)
# d1 = uniform(229.44,954.72)
#d2 = uniform(d1 + 487.610 , 1652.93)

In [18]:
#Conversion for scaling

MeV = 1 
GeV = 1000*MeV
fm  = 1
c   = 1
metres = (10**15)*fm
km = 1000*metres
second = 2.99*(10**8)*metres
kg = (second**2)/((1.6*(10**(-13)))*((metres)**2))
G = 6.67*10**(-11)*((metres)**3)/(kg*(second)**2)
mbar = 1.66*(10**-27)*kg

In [19]:
import numpy as np
import scipy
import matplotlib.pyplot as plot
import math
from scipy.integrate import odeint
from scipy.stats import uniform

In [20]:
def piecewise_polytrope_SFHx(p, p1, p2, k1, GA1, GA2, GA3):
    
    k2 = p1/((p1/k1)**(1/GA1) + p1/(GA1-1) - p1/(GA2-1))**GA2
    k3 = p2/((p2/k2)**(1/GA2) + p2/(GA2-1) - p2/(GA3-1))**GA3

    condition1 = (p < p1)
    condition2 = (p >= p1) & (p < p2)
    condition3 = (p >= p2) 
    condition = [condition1, condition2, condition3]
    Polytrope1 = lambda p: (p/k1)**(1/GA1) + (p/(GA1-1))
    Polytrope2 = lambda p: (p/k2)**(1/GA2) + (p/(GA2-1))
    Polytrope3 = lambda p: (p/k3)**(1/GA3) + (p/(GA3-1))
    function_list = [Polytrope1 , Polytrope2 , Polytrope3]
    d = (np.piecewise(p, condition, function_list))
    
    return (d)

In [26]:
#SFHx
dp = 10**-12
mbar = 1.66*(10**-27)*kg
pi = math.pi
Rs = ((2*G*(1.98847*(10**30)*kg))/((c**2)))
K1 =  ((4*pi*(Rs**3))/(1.98847*((10**30)*kg)*(c**2)))*(GeV/(fm)**3)

class TOV_Solver:
    def __init__(self,Pc,p1,p2,k1,GA1,GA2,GA3):
        self.Pc = Pc
        self.p1 = p1
        self.p2 = p2
        self.k1 = k1
        self.GA1 = GA1
        self.GA2 = GA2
        self.GA3 = GA3
        
    def R0(self,Pc,GA,k):
            dc = (mbar*(self.Pc*(GeV)/k)**(1/GA) + (self.Pc*(GeV)/(GA - 1)))/(GeV)
            P = self.Pc - dp 
            d = (mbar*(P*GeV/k)**(1/GA) +(P*GeV/(GA - 1)))/GeV
            nc = (((Pc*GeV)/k)**(1/GA))/GeV
            n = (((P*GeV)/k)**(1/GA))/GeV
            return  (((3/(2*pi*(3*self.Pc + dc)))*((math.log((dc + self.Pc)*n) - ((math.log((d + P)*nc)))))))**(0.5)

    def M0(self):
            dc = (mbar*(Pc*(GeV)/k)**(1/GA) + (Pc*(GeV)/(GA - 1)))/(GeV)
            P = Pc - dp
            d = (mbar*(P*GeV/k)**(1/GA) +(P*GeV/(GA - 1)))/GeV
            nc = (((Pc*GeV)/k)**(1/GA))/GeV
            n = (((P*GeV)/k)**(1/GA))/GeV
            return ((((((math.log((dc + Pc)*n) - ((math.log((d + P)*nc)))))*(2*dc/(3*Pc + dc))))))*R0(Pc,dp)
        
    def f(self):
        m,r = x
        d = piecewise_polytrope_SFHx(P,self.p1,self.p2,self.k1,self.GA1,self.GA2,self.GA3)/GeV
        dmdp = -(2*K1*(r**3)*d*(r-m))/((d + P)*(m + K1*P*(r**3)))
        drdp = -(2*r*(r - m))/((d + P)*(m + K1*P*(r**3)))
        return [dmdp , drdp]  
            
            
    def compute_MR(self):
        
                 
        Ro =  R0(Pc = self.Pc/GeV,  dp = dp)
        Mo =  M0(Pc = self.Pc/GeV,  dp = dp)
        R = []
        M = []
        x_i = [Mo,Ro]
                         
        P0 = Pc/GeV - dp
            
        P1 = np.linspace(1e-12, P0, 1000)
        solve = odeint(f,x_i, P1[::-1])
                          
        m = solve[:,0]
        r = solve[:,1]

        M = (m[-1])
        R = (r[-1])*(Rs/km)
        return "M:",M ,"R:" ,R 
    

In [27]:
TOV_Solver.compute_MR(1000)

NameError: name 'R0' is not defined